In [19]:
import shleem

In [20]:
shleem_test_server = shleem.mongodb.server("shleem_test_server")

In [21]:
client = shleem_test_server._get_connection()

In [22]:
example_data_col = client['shleem_test']['example_data_collection']

In [23]:
from bson.objectid import ObjectId

In [24]:
doc_with_subdocs = example_data_col.find_one({'_id': ObjectId("59e8a02a297efbb3162667f1")})

In [25]:
# doc_with_subdocs

In [26]:
# sorted(list(doc_with_subdocs.keys()))

In [27]:
import csv

In [28]:
from strct.dicts import flatten_dict

In [29]:
with open('test_dump.csv', 'w+') as dump_csv:
    fieldnames = sorted(list(flatten_dict(doc_with_subdocs).keys()))
    writer = csv.DictWriter(dump_csv, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow(flatten_dict(doc_with_subdocs))

## Dump to json

In [35]:
from bson.json_util import (
    dumps, 
    loads,
)

In [31]:
dumps(doc_with_subdocs)

'{"_id": {"$oid": "59e8a02a297efbb3162667f1"}, "address": {"building": "469", "coord": [-73.961704, 40.662942], "street": "Flatbush Avenue", "zipcode": "11225"}, "borough": "Brooklyn", "cuisine": "Hamburgers", "grades": [{"date": {"$date": 1419897600000}, "grade": "A", "score": 8}, {"date": {"$date": 1404172800000}, "grade": "B", "score": 23}, {"date": {"$date": 1367280000000}, "grade": "A", "score": 12}, {"date": {"$date": 1336435200000}, "grade": "A", "score": 12}], "name": "Wendy\'S", "restaurant_id": "30112340"}'

In [45]:
with open('test_single_dump.json', 'w+') as dump_json:
    dump_json.write('[\n')
    dump_json.write(dumps(doc_with_subdocs))
    dump_json.write('\n]')

In [46]:
with open('test_single_dump.json', 'r') as dump_json:
    result = loads(dump_json.read())
    print(result)

[{'_id': ObjectId('59e8a02a297efbb3162667f1'), 'address': {'building': '469', 'coord': [-73.961704, 40.662942], 'street': 'Flatbush Avenue', 'zipcode': '11225'}, 'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2014, 7, 1, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'B', 'score': 23}, {'date': datetime.datetime(2013, 4, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2012, 5, 8, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 12}], 'name': "Wendy'S", 'restaurant_id': '30112340'}]


In [40]:
result[0]

{'_id': ObjectId('59e8a02a297efbb3162667f1'),
 'address': {'building': '469',
  'coord': [-73.961704, 40.662942],
  'street': 'Flatbush Avenue',
  'zipcode': '11225'},
 'borough': 'Brooklyn',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>),
   'grade': 'A',
   'score': 8},
  {'date': datetime.datetime(2014, 7, 1, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>),
   'grade': 'B',
   'score': 23},
  {'date': datetime.datetime(2013, 4, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>),
   'grade': 'A',
   'score': 12},
  {'date': datetime.datetime(2012, 5, 8, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>),
   'grade': 'A',
   'score': 12}],
 'name': "Wendy'S",
 'restaurant_id': '30112340'}

## Cursor probing

In [13]:
some_3_docs = example_data_col.find({
    '_id': {'$in': [
        ObjectId("59e8a02a297efbb3162667f1"), ObjectId("59e8a02a297efbb3162667f2"), 
        ObjectId("59e8a02a297efbb3162667f3")
    ]}
})

In [76]:
type(first_doc)

dict

In [14]:
first_doc = some_3_docs.next()
print((0, first_doc['_id']))

(0, ObjectId('59e8a02a297efbb3162667f1'))


In [15]:
for i, doc in enumerate(some_3_docs):
    print((i, doc['_id']))

(0, ObjectId('59e8a02a297efbb3162667f2'))
(1, ObjectId('59e8a02a297efbb3162667f3'))


In [71]:
some_3_docs = example_data_col.find({
    '_id': {'$in': [
        ObjectId("59e8a02a297efbb3162667f1"), ObjectId("59e8a02a297efbb3162667f2"), 
        ObjectId("59e8a02a297efbb3162667f3")
    ]}
})

In [72]:
file_path = 'test_multiple_dump.json'

In [73]:
doc_cursor = some_3_docs

In [74]:
with open(file_path, 'w+') as dump_json:
    dump_json.write('[\n')
    dump_json.write(dumps(doc_cursor.next()))
    for doc in doc_cursor:
        dump_json.write(',\n')
        dump_json.write(dumps(doc))
    dump_json.write('\n]')

In [83]:
from json import JSONDecodeError

In [86]:
with open(file_path, 'r') as load_json:
    line = load_json.readline()
    while line:
        if line not in ('[\n', ']'):  # ignore start and end of array
            try:  # skip trailing , and one \n
                print(loads(line[:-2]))
            except JSONDecodeError:  # last line has no , so just \n
              print(loads(line[:-1]))  
        line = load_json.readline()

{'_id': ObjectId('59e8a02a297efbb3162667f1'), 'address': {'building': '469', 'coord': [-73.961704, 40.662942], 'street': 'Flatbush Avenue', 'zipcode': '11225'}, 'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2014, 7, 1, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'B', 'score': 23}, {'date': datetime.datetime(2013, 4, 30, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2012, 5, 8, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x105ae1be0>), 'grade': 'A', 'score': 12}], 'name': "Wendy'S", 'restaurant_id': '30112340'}
{'_id': ObjectId('59e8a02a297efbb3162667f2'), 'address': {'building': '2780', 'coord': [-73.98241999999999, 40.579505], 'street': 'Stillwell Avenue', 'zipcode': '11224'}, 'borough': 'Brooklyn', 'cuisine': 

## Lazy iterator

## Finding nice queries

In [17]:
len(list(example_data_col.find({
    "borough": "Brooklyn",
    "cuisine": "Hamburgers",    
})))

102